In [1]:
# Импорт библиотек
import os
import time
import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 20

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 10,
    "kolichestvo": 1,
    "Visota": 15,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}

syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 20
pkm_pgu_tol = 10**-2




In [2]:
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter

PKM_zaryad = True
PKM_razryad = False

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm=20
pkm_pgu_tol=10**-2
CCGT=Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

2 100
Тепла от ПКМ хватает на теплофикацию
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 3
Время 1 итерации расчета КУ+ТУ:---  10.2 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 3
Время 2 итерации расчета КУ+ТУ:---  18.4 сек. ---
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 3
Время 3 итерации расчета КУ+ТУ:---  25.4 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 3
Pnd_it [0.6878, 0.68877, 0.68795, 0.68719]
Pvd_it [8.41904, 8.3093, 8.26109, 8.23988]
Max_error_P 0.5132928446927196
Max_error_G 0.01853979963482674
fin КУ и ТУ:--- 25.4 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 27.0 сек. ---
2 100
Тепла от ПКМ хватает на теплофикацию
Время 1 итерации расчета КУ+ТУ:---  5.5 сек. ---
Время 2 итерации расчета КУ+ТУ:---  8.1 сек. ---
Время 3 итерации расчета КУ+ТУ:---  10.1 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 3
Pnd_i

In [3]:
water_streams

,T,P,H,G,S,X
AIR,8.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,509.154153,8.255158,3419.149183,63.684797,NaN,NaN
IVD-PEVD,300.132493,8.603947,2749.383528,63.684797,NaN,NaN
EVD-IVD,298.192837,8.603947,1334.654151,63.684797,NaN,NaN
PEN-EVD,166.067182,8.603947,706.366066,63.684797,NaN,NaN
BND-PEN,164.610650,0.694195,695.538776,63.684797,NaN,NaN
PPND-DROSND,177.899841,0.688308,2795.226348,15.415346,NaN,NaN
IND-PPND,164.610649,0.694195,2762.400883,15.415346,NaN,NaN
GPK-IND,163.109896,0.694195,689.014520,79.100143,NaN,NaN
GPK-REC,163.109896,0.694195,689.014520,87.446038,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3687.984095,1.496742,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3030.242194,1.496742,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2555.053441,1.496742,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,791.135714,1.496742,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1348.041621,0.595338,0.0,0.0,0.168668,0.0,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
heaters

,Qw,Qg,KPD
PEVD,42655.956133,43086.824377,0.99
IVD,90097.261455,91007.334803,0.99
EVD,40012.427049,40416.592979,0.99
PPND,505.968551,511.079344,0.99
IND,32378.015459,32705.066121,0.99
GPK,38237.351907,38623.587784,0.99
SP2,0.000000,NaN,0.00
SP1,0.000000,NaN,0.00
OD,0.000000,NaN,0.00
Strans,4307.655167,NaN,NaN


In [6]:
accumulation

,Qw,T,V,Qnr
ASW,NaN,NaN,NaN,NaN
PKM,NaN,NaN,NaN,NaN


In [7]:
from PKM import PKM_all

Gst = 1
pkm = PKM_all.calc(True,gas_streams,syngas_streams,water_streams,water_streams0,heaters, Gst)

TypeError: calc() takes 6 positional arguments but 7 were given

In [ ]:
accumulation